# Data Processing

Notebook for processing data files into the correct form for mapping. We'll need 4 files for mapping:

1. Election Precinct shape files
2. Congressional District shape files
3. Primary votes at the Precinct level
4. Primary votes aggregated to the Congressional District level (+ additional columns)

In [1]:
import geopandas as gpd
import pandas as pd
import sys

In [2]:
# repo_path = "/[yourpath]/nyc-primary-2025/"

sys.path.insert(0, repo_path + "/src")
sys.path.insert(0, repo_path + "/data")

## Combine Geo files and de-duplicate

In [3]:
# read in shape files
gdfa = gpd.read_file(repo_path + "/data/nyed_25b/nyed.shp")
gdfc = gpd.read_file(repo_path + "/data/nycg_25b/nycg.shp")

In [4]:
# join geo files by overlaying election precincts on congressional districts
udf = gdfc.overlay(gdfa, how="union")

/Users/emilyschuch/.pyenv/versions/dsenv/lib/python3.12/site-packages/geopandas/tools/overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 6059 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


In [5]:
# rejoin congressional districts
intdf = gpd.sjoin(udf, gdfc, how="left")
intdf.shape

(7165, 11)

In [6]:
# rename columns
intdf = intdf[["CongDist_left", "ElectDist", "geometry"]]
intdf.columns = ["cong_dist", "precinct_id", "geometry"]
intdf.columns

Index(['cong_dist', 'precinct_id', 'geometry'], dtype='object')

In [7]:
# de-duplicate overlapping geometry
intdf = intdf.dissolve(by=["cong_dist", "precinct_id"], aggfunc='first').reset_index()
intdf["geometry"] = intdf.normalize()
intdf = intdf.drop_duplicates()
dist = intdf[["cong_dist", "precinct_id"]]
dist = dist.groupby("precinct_id")["cong_dist"].agg("first").reset_index()

In [8]:
dist.shape, intdf.shape

((4264, 2), (4764, 3))

In [9]:
dist.columns

Index(['precinct_id', 'cong_dist'], dtype='object')

## Merge Vote data with Election Districts & Reshape

In [10]:
# read in voting data
vdf = pd.read_csv(repo_path + "/data/votes.csv")

In [11]:
# create standardized precint id, which is a concatenation of Assembly District id and Election District id
# the `ElectDist` field in the geo files is already in the format but needs to be converted to a string

dist["precinct_id"] = dist["precinct_id"].map(lambda x: str(round(x)))

vdf["ad_id"] = vdf["AD"].map(lambda x: x.split("-")[0].replace("AD", ""))
vdf["ed_id"] = vdf["election_district"].map(lambda x: x.replace("ED ", "").zfill(3))
vdf["precinct_id"] = vdf[["ad_id", "ed_id"]].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [12]:
# merge voting data and geo ids without geometry
votes = pd.merge(vdf, dist, how="left", on="precinct_id")

In [13]:
votes.columns

Index(['AD', 'election_district', 'pct_reported', 'zohran_kwame_mamdani',
       'scott_m_stringer', 'selma_k_bartholomew', 'zellnor_myrie',
       'adrienne_e_adams', 'andrew_m_cuomo', 'jessica_ramos',
       'whitney_r_tilson', 'michael_blake', 'brad_lander',
       'paperboy_love_prince', 'write_in', 'ad_id', 'ed_id', 'precinct_id',
       'cong_dist'],
      dtype='object')

## Add Additional Columns

In [14]:
candidates = ["zohran_kwame_mamdani", "scott_m_stringer", "selma_k_bartholomew", "zellnor_myrie",
              "adrienne_e_adams", "andrew_m_cuomo", "jessica_ramos", "whitney_r_tilson",
              "michael_blake", "brad_lander", "paperboy_love_prince", "write_in"]

top_candidates = ["zohran_kwame_mamdani", "brad_lander", "andrew_m_cuomo"]
else_candidates = [c for c in candidates if c not in top_candidates]
progressive_candidates = ["zohran_kwame_mamdani", "brad_lander"]


votes["total"] = votes[candidates].apply(lambda row: sum(row), axis=1)
votes["progressive"] = votes[progressive_candidates].apply(lambda row: sum(row), axis=1)
votes["else"] = votes[else_candidates].apply(lambda row: sum(row), axis=1)

In [15]:
votes[candidates + ["total", "progressive", "else"]].sum().sort_values(ascending=False)

total                   993546
progressive             544654
zohran_kwame_mamdani    432305
andrew_m_cuomo          361840
brad_lander             112349
else                     87052
adrienne_e_adams         40953
scott_m_stringer         16387
zellnor_myrie             9870
whitney_r_tilson          7828
michael_blake             3992
jessica_ramos             3862
paperboy_love_prince      1417
write_in                  1415
selma_k_bartholomew       1328
dtype: int64

## Recombine Geo Data

In [16]:
votes_ge = pd.merge(dist, votes, how="left", on="precinct_id")

In [17]:
# filter & rename columns
votes_ge = votes_ge[["precinct_id", "cong_dist_x", "total", "progressive", "else"] + candidates]
votes_ge.columns = ["precinct_id", "cong_dist"] + list(votes_ge.columns)[2:]

In [18]:
# format id numbers to strings
for col in "cong_dist", "precinct_id":
    votes_ge[col] = [str(round(int(x))) for x in votes_ge[col]]

In [19]:
votes_ge.columns

Index(['precinct_id', 'cong_dist', 'total', 'progressive', 'else',
       'zohran_kwame_mamdani', 'scott_m_stringer', 'selma_k_bartholomew',
       'zellnor_myrie', 'adrienne_e_adams', 'andrew_m_cuomo', 'jessica_ramos',
       'whitney_r_tilson', 'michael_blake', 'brad_lander',
       'paperboy_love_prince', 'write_in'],
      dtype='object')

## Aggregate Congressional District Data

In [20]:
import requests
import bs4

In [21]:
# pull data from the cook political report
url ="https://ballotpedia.org/The_Cook_Political_Report%27s_Partisan_Voter_Index"
req = requests.get(url)
req.status_code

200

In [22]:
soup = bs4.BeautifulSoup(req.text, 'html.parser')

print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('id'))

Classes of each table:
None
None
None
dt-06a0aa5d
dt-a41d46c1
dt-62b25c19
dt-e2132823
dt-0849bbcb
None
None


In [23]:
# the data we want is in this table
table_id = "dt-06a0aa5d"
table = soup.find('table', id=table_id)

In [24]:
# define of the dataframe
df = pd.DataFrame(columns=['district', 'incumbent', 'pvi'])

# collect the data
for row in table.tbody.find_all('tr'):    
    # find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        district = columns[0].text.strip()
        incumbent = columns[1].text.strip()
        pvi = columns[2].text.strip()

        new_row = pd.DataFrame({'district': district,  'incumbent': incumbent, 'pvi': pvi}, index=[0])

        df = pd.concat([df, new_row], ignore_index=True)

In [25]:
# isolate New York
df["state"] = df["district"].map(lambda x: x.split("'")[0])
df["cong_dist"] = df["district"].map(lambda x: x.split("'")[1].strip("s "))

dfny = df[df.state == "New York"]
dfny

,district,incumbent,pvi,state,cong_dist
272,New York's 1st,Nick LaLota,R+4,New York,1st
273,New York's 2nd,Andrew Garbarino,R+6,New York,2nd
274,New York's 3rd,Tom Suozzi,EVEN,New York,3rd
275,New York's 4th,Laura Gillen,D+2,New York,4th
276,New York's 5th,Gregory Meeks,D+24,New York,5th
277,New York's 6th,Grace Meng,D+6,New York,6th
278,New York's 7th,Nydia Velázquez,D+25,New York,7th
279,New York's 8th,Hakeem Jeffries,D+24,New York,8th
280,New York's 9th,Yvette Clarke,D+22,New York,9th
281,New York's 10th,Dan Goldman,D+32,New York,10th


In [26]:
# formatting & parsing
for sfx in "st", "nd", "rd", "th":
    dfny["cong_dist"] = dfny["cong_dist"].map(lambda x: x.replace(sfx, ""))

/var/folders/97/_bpy961n2hq12htctyl45qsw0000gn/T/ipykernel_96563/1520771818.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfny["cong_dist"] = dfny["cong_dist"].map(lambda x: x.replace(sfx, ""))


In [28]:
repubs = [str(x) for x in [1, 2, 11, 17, 21, 23, 24]]

dfny["party"] = dfny["cong_dist"].map(lambda x: "R" if x in repubs else "D")

/var/folders/97/_bpy961n2hq12htctyl45qsw0000gn/T/ipykernel_96563/1461564359.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfny["party"] = dfny["cong_dist"].map(lambda x: "R" if x in repubs else "D")


In [33]:
dfny["pvi_party"] = dfny["pvi"].map(lambda x: x.split("+")[0])
dfny["pvi_value"] = dfny["pvi"].map(lambda x: 0 if x=="EVEN" else int(x.split("+")[1]))
dfny["pvi_value"] = dfny.apply(lambda x: -x.pvi_value if x.pvi_party=="R" else x.pvi_value, axis=1)

/var/folders/97/_bpy961n2hq12htctyl45qsw0000gn/T/ipykernel_96563/2373579328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfny["pvi_party"] = dfny["pvi"].map(lambda x: x.split("+")[0])
/var/folders/97/_bpy961n2hq12htctyl45qsw0000gn/T/ipykernel_96563/2373579328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfny["pvi_value"] = dfny["pvi"].map(lambda x: 0 if x=="EVEN" else int(x.split("+")[1]))
/var/folders/97/_bpy961n2hq12htctyl45qsw0000gn/T/ipykernel_96563/2373579328.py:3: SettingWithCopyWarning:

In [34]:
dfny

,district,incumbent,pvi,state,cong_dist,party,pvi_party,pvi_value
272,New York's 1st,Nick LaLota,R+4,New York,1,R,R,-4
273,New York's 2nd,Andrew Garbarino,R+6,New York,2,R,R,-6
274,New York's 3rd,Tom Suozzi,EVEN,New York,3,D,EVEN,0
275,New York's 4th,Laura Gillen,D+2,New York,4,D,D,2
276,New York's 5th,Gregory Meeks,D+24,New York,5,D,D,24
277,New York's 6th,Grace Meng,D+6,New York,6,D,D,6
278,New York's 7th,Nydia Velázquez,D+25,New York,7,D,D,25
279,New York's 8th,Hakeem Jeffries,D+24,New York,8,D,D,24
280,New York's 9th,Yvette Clarke,D+22,New York,9,D,D,22
281,New York's 10th,Dan Goldman,D+32,New York,10,D,D,32


In [53]:
dfny.to_csv(repo_path + "data/cong_data.csv", index=False)

## Add Additional District Info

## Format Shape Files & Raw Data

In [45]:
gdfa.head()

,ElectDist,Shape_Leng,Shape_Area,geometry,precinct_id,precinct
0,23001,24593.971343,2.779129e+07,"POLYGON ((1006386.279 144120.655, 1006261.842 ...",23001,23001
1,23002,15531.629161,9.753401e+06,"POLYGON ((1009207.47 145106.094, 1009147.042 1...",23002,23002
2,23003,41787.652835,3.452959e+07,"POLYGON ((1022349.755 145817.949, 1022404.165 ...",23003,23003
3,23004,13616.939280,8.166450e+06,"POLYGON ((1025161.154 147197.743, 1025055.81 1...",23004,23004
4,23005,10698.547734,5.077745e+06,"POLYGON ((1026261.273 147727.118, 1026236.365 ...",23005,23005


In [46]:
gdfa["precinct"] = gdfa.ElectDist.map(lambda x: str(x))

In [47]:
vdf.head()

,AD,election_district,pct_reported,zohran_kwame_mamdani,scott_m_stringer,selma_k_bartholomew,zellnor_myrie,adrienne_e_adams,andrew_m_cuomo,jessica_ramos,whitney_r_tilson,michael_blake,brad_lander,paperboy_love_prince,write_in,ad_id,ed_id,precinct_id
0,AD37-1,ED 71,80.0,0,0,0,0,0,0,0,0,0,0,0,0,37,071,37071
1,AD61-1,ED 60,99.0,141,6,0,5,11,75,1,3,0,53,0,1,61,060,61060
2,AD61-1,ED 61,99.0,148,9,0,6,11,175,4,16,1,70,0,0,61,061,61061
3,AD61-1,ED 62,99.0,106,14,0,3,6,141,1,5,1,64,0,1,61,062,61062
4,AD61-1,ED 63,99.0,118,20,0,7,10,166,0,13,1,71,0,0,61,063,61063


In [48]:
votes.head()

,AD,election_district,pct_reported,zohran_kwame_mamdani,scott_m_stringer,selma_k_bartholomew,zellnor_myrie,adrienne_e_adams,andrew_m_cuomo,jessica_ramos,...,brad_lander,paperboy_love_prince,write_in,ad_id,ed_id,precinct_id,cong_dist,total,progressive,else
0,AD37-1,ED 71,80.0,0,0,0,0,0,0,0,...,0,0,0,37,071,37071,7.0,0,0,0
1,AD61-1,ED 60,99.0,141,6,0,5,11,75,1,...,53,0,1,61,060,61060,10.0,296,194,27
2,AD61-1,ED 61,99.0,148,9,0,6,11,175,4,...,70,0,0,61,061,61061,10.0,440,218,47
3,AD61-1,ED 62,99.0,106,14,0,3,6,141,1,...,64,0,1,61,062,61062,10.0,342,170,31
4,AD61-1,ED 63,99.0,118,20,0,7,10,166,0,...,71,0,0,61,063,61063,10.0,406,189,51


In [41]:
type(votes.precinct_id[0])

str

In [49]:
gdfa.to_file(repo_path + "/data/election_districts.shp")

/var/folders/97/_bpy961n2hq12htctyl45qsw0000gn/T/ipykernel_96563/1358178982.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdfa.to_file(repo_path + "/data/election_districts.shp")
/Users/emilyschuch/.pyenv/versions/dsenv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'precinct_id' to 'precinct_i'
  ogr_write(
/Users/emilyschuch/.pyenv/versions/dsenv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 197031970.13299999 of field Shape_Area of feature 552 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/Users/emilyschuch/.pyenv/versions/dsenv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Value 112675897.639 of field Shape_Area of feature 2639 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


In [51]:
votes["ElectDist"] = votes.precinct_id.map(lambda x: int(x))

In [52]:
votes.to_csv(repo_path + "/data/votes_precinct.csv", index=False)